In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import options as opt
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
def heston_model(S0, v0, r, rho, kappa, theta, sigma, T, M):
    N=int(round(T*252))
    dt = T / N
    mu = np.array([0, 0])
    cov = np.array([[1, rho], [rho, 1]])

    S = np.full(shape=(N + 1, M), fill_value=S0)  # Initialisation des prix à S0
    v = np.full(shape=(N + 1, M), fill_value=v0)  # Initialisation des variances à v0

    # Générer des échantillons de variables aléatoires multivariées
    Z = np.random.multivariate_normal(mu, cov, (N, M))

    for i in range(1, N + 1):
        S[i] = S[i - 1] * np.exp((r - 0.5 * v[i - 1]) * dt + np.sqrt(v[i - 1] * dt) * Z[i - 1, :, 0])
        v[i] = np.maximum(v[i - 1] + kappa * (theta - v[i - 1]) * dt + sigma * np.sqrt(v[i - 1] * dt) * Z[i - 1, :, 1], 0)

    return S, v

In [27]:
S1, v1 = heston_model(S0=100, v0=0.2, r=0.02, rho=1, kappa=0.5, theta=1, sigma=0, T=1, M=1000)

In [28]:
pd.DataFrame(S1)

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
1,103,99,99,100,101,96,100,98,101,100,...,103,96,100,104,101,96,102,100,104,101
2,104,97,101,97,100,97,102,94,97,97,...,105,97,98,105,102,95,98,99,101,103
3,102,97,99,98,103,97,105,97,95,96,...,108,99,96,108,101,93,95,104,102,95
4,106,92,96,103,102,100,106,93,89,94,...,109,98,96,107,104,91,99,103,98,94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,43,4,7,0
249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,39,4,6,0
250,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,41,3,5,0
251,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,39,2,4,0


In [13]:
def heston_model_antithetic(S0, v0, r, rho, kappa, theta, sigma, T, M):
    N=int(round(T*252))
    dt = T / N
    M=int(round(M/2))

    mu = np.array([0, 0])
    cov1 = np.array([[1, rho], [rho, 1]])
    cov2 = np.array([[-1, rho], [rho, 1]])

    S = np.full(shape=(N + 1, M), fill_value=S0)  # Initialisation des prix à S0
    v = np.full(shape=(N + 1, M), fill_value=v0)  # Initialisation des variances à v0

    # Générer des échantillons de variables aléatoires multivariées
    Z1 = np.random.multivariate_normal(mu, cov1, (N, M))
    Z2 = np.random.multivariate_normal(mu, cov2, (N, M))

    Z = Z1 + Z2

    for i in range(1, N + 1):
        S[i] = S[i - 1] * np.exp((r - 0.5 * v[i - 1]) * dt + np.sqrt(v[i - 1] * dt) * Z[i - 1, :, 0])
        v[i] = np.maximum(v[i - 1] + kappa * (theta - v[i - 1]) * dt + sigma * np.sqrt(v[i - 1] * dt) * Z[i - 1, :, 1], 0)

    return S, v

In [14]:
S2, v2 = heston_model_antithetic(S0=100, v0=0.2, r=0.02, rho=1, kappa=0.5, theta=1, sigma=0.2, T=1, M=1000)

C:\Users\Bouchereau Killian\AppData\Local\Temp\ipykernel_1432\3985500367.py:15: RuntimeWarning: covariance is not positive-semidefinite.
  Z2 = np.random.multivariate_normal(mu, cov2, (N, M))


In [15]:
S2.shape

(253, 500)